# Notebook Instructions

1. If you are new to Jupyter notebooks, please go through this introductory manual <a href='https://quantra.quantinsti.com/quantra-notebook' target="_blank">here</a>.
1. Any changes made in this notebook would be lost after you close the browser window. **You can download the notebook to save your work on your PC.**
1. Before running this notebook on your local PC:<br>
i.  You need to set up a Python environment and the relevant packages on your local PC. To do so, go through the section on "**Run Codes Locally on Your Machine**" in the course.<br>
ii. You need to **download the zip file available in the last unit** of this course. The zip file contains the data files and/or python modules that might be required to run this notebook.

# Trade Level Analytics of ML Based Spread Trading Strategy

In the previous notebook, we have backtested the bull call spread strategy. In this notebook, we will use the backtest results and perform trade level analytics.

The metrics such as average PnL per trade, win & loss percentage of trades, average holding period, and profit factor help us better understand the trades generated over the backtest period.

The notebook is structured as follows:

1. [Read the Data](#read)
2. [Tradesheet](#tradesheet)
3. [Different Trade Level Analytics](#trade)<br>
   3.1. [Profit and Loss](#pnl)<br>
   3.2. [Win Percentage](#win)<br>
   3.3. [Average PnL Per Trade](#avg)<br>
   3.4. [Average Trade Duration](#time)<br>
   3.5. [Profit Factor](#profit)<br>

## Import Libraries

In [1]:
# For data manipulation
import pandas as pd
import numpy as np

# Ignore warnings
import warnings 
warnings.filterwarnings('ignore')

# For plotting
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

<a id='read'></a>
## Read the Data

In the previous section, the ML based bull call spread strategy was backtested and the round trips were generated. Import the round trips data from `round_trips_call_spread.csv` present in the `data_modules` folder. 

In [2]:
# Import the round trip details
round_trips_details = pd.read_csv('../data_modules/round_trips_call_spread.csv')

<a id='tradesheet'></a>
## Trade Level Analytics

In [3]:
# Lot Size
lot_size = 100

# Profit and Loss of trades
round_trips_details['pnl'] = round_trips_details['position'] * \
    (round_trips_details['exit_price']-round_trips_details['entry_price'])

# Create a dataframe for storing trades
trades = pd.DataFrame()

# Groupby entry date
trades_group = round_trips_details.groupby('entry_date')

# Group trades from round_trips
trades['Entry_Date'] = trades_group['entry_date'].first()
trades['Exit_Date'] = trades_group['exit_date'].first()
trades['Exit_Type'] = trades_group['exit_type'].first()

# Calculate PnL for the strategy for 1 lot
trades['PnL'] = trades_group.pnl.sum() * lot_size

# Calculate turnover for trades
trades['Turnover'] = (trades_group['exit_price'].sum() +
                      trades_group['entry_price'].sum()) * lot_size

trades['PnL_post_trading_costs_slippages'] = trades['PnL']
# Reset index
trades.reset_index(inplace=True)
trades.head()

,entry_date,Entry_Date,Exit_Date,Exit_Type,PnL,Turnover,PnL_post_trading_costs_slippages
0,2018-07-20,2018-07-20,2018-07-24,TP,900.0,4510.0,900.0
1,2018-07-26,2018-07-26,2018-07-27,SL,-740.0,1110.0,-740.0
2,2018-07-30,2018-07-30,2018-07-31,Expiry or Signal Based,869.0,2479.0,869.0
3,2018-08-01,2018-08-01,2018-08-02,Expiry or Signal Based,-162.0,6352.0,-162.0
4,2018-08-03,2018-08-03,2018-08-07,Expiry or Signal Based,1145.0,6485.0,1145.0


<a id='trade'></a>
## Different Trade Level Analytics

Create a dataframe `analytics` that stores different trade level analytics of ML model's backtest results

In [4]:
# Dataframe 'analytics'
analytics = pd.DataFrame(index=['ML_Model'])

<a id='pnl'></a>
### Profit and Loss 

In the `trades` dataframe, the column `Pnl_post_trading_costs_slippages` represents the net profit and loss per unit of the underlying including the trading and slippage costs. Since options are traded in lot sizes, the total profit and loss would be a multiple of `Pnl_post_trading_costs_slippages` and the lot size of the contract.



In [5]:
# Assume lot size as 100
lot_size = 100

# Calculate total PnL and store in 'analytics'
analytics['Total_PnL'] = trades['PnL_post_trading_costs_slippages'].sum()

# Print the value
print("Total_PnL:", round(analytics['Total_PnL'][0], 2))

Total_PnL: 43223.0


The total profit and loss doesn't give detailed information such as how many trades contributed to the profit. 
<a id='win'></a>
### Win Percentage
The win percentage or win rate is an essential metric. It represents the percentage of trades which were profitable out of the total trades, to determine a strategy's success. A win rate above 50% is usually favourable.

To calculate the win percentage, divide the number of profitable trades  by the total number of trades as shown below:


$$ Win~Rate = \frac{No.~of~Winning~Trades}{Total~No.~of~Trades} *100$$

Similarly, we can also calculate the loss percentage by dividing the number of losing trades by the total number of trades. 

In [6]:
# First, let's find the total number of trades we have made from the butterfly spread strategy
analytics['Total_Trades'] = len(trades)

# Winning trades
analytics['Winners'] = len(trades.loc[trades.PnL >= 0])

# Losing trades
analytics['Losers'] = len(trades.loc[trades.PnL < 0])

# Win percentage
analytics['Win_Percentage'] = 100 * \
    (analytics['Winners']/analytics['Total_Trades'])

# Loss percentage
analytics['Loss_Percentage'] = 100 * \
    (analytics['Losers']/analytics['Total_Trades'])

# Print the 'analytics' dataframe
analytics.T

,ML_Model
Total_PnL,43223.000000
Total_Trades,268.000000
Winners,153.000000
Losers,115.000000
Win_Percentage,57.089552
Loss_Percentage,42.910448


<a id='avg'></a>
### Average PnL Per Trade
The average PnL per trade is used to find the average amount that you can expect to gain or lose on each trade. This metric tells us how much impact a winning or losing trade might have. In general, we want the average loss per losing trade to be as low as possible and the average profit per winning trade as high as possible. 

For example, if your average loss per losing trade is 3x your average profit per winning trade, a single loser will wipe out the profits of 3 winners. You can determine the average profit per winning trade by dividing the sum amount of all the profits by the number of winning trades.

$$ Average~Profit~Per~Winning~Trade = \frac{Total~Profit~made~by~all~Winners}{No.~of~Winning~Trades} $$

Similarly, you can find the average loss per losing trade by dividing the sum of all the losses by the number of losing trades.

In [7]:
# Avg profit per trade of winners
analytics['per_trade_profit_winners'] = trades.loc[trades.PnL >
                                                   0].PnL.mean()

# Avg loss per trade of losers
analytics['per_trade_loss_losers'] = abs(
    trades.loc[trades.PnL < 0].PnL.mean())

# Print the 'analytics' dataframe
round(analytics.T, 2)

,ML_Model
Total_PnL,43223.00
Total_Trades,268.00
Winners,153.00
Losers,115.00
Win_Percentage,57.09
Loss_Percentage,42.91
per_trade_profit_winners,2084.05
per_trade_loss_losers,2143.13


<a id='time'></a>
### Average Trade Duration
The average trade duration, also known as the average holding period, is the average amount of time you remain in a trade.

To calculate the average trade duration, we first calculate the holding period per trade i.e. `Exit_Date` - `Entry_Date`. Next, we calculate the mean of the holding time using the `mean()` method. 

In [8]:
# Calculate holding period for each trade
trades['Entry_Date'] = pd.to_datetime(trades['Entry_Date'], format='%Y-%m-%d')
trades['Exit_Date'] = pd.to_datetime(trades['Exit_Date'], format='%Y-%m-%d')
trades['holding_period'] = trades['Exit_Date'] - trades['Entry_Date']

# Calculate the average holding period in days and store in 'analytics'
analytics['Average holding time'] = trades['holding_period'].mean().days

# Print the 'analytics' dataframe
round(analytics.T, 2)

,ML_Model
Total_PnL,43223.00
Total_Trades,268.00
Winners,153.00
Losers,115.00
Win_Percentage,57.09
Loss_Percentage,42.91
per_trade_profit_winners,2084.05
per_trade_loss_losers,2143.13
Average holding time,3.00


<a id='profit'></a>
### Profit Factor
The profit factor measures the amount of money made against the money lost while trading. 
It is the ratio of the sum of profit to the sum of loss. It can also be calculated with the following formula: 

$$ Profit~Factor = \frac{~Win~Percentage~*~Average~Profit~Per~Winning~Trade}{~Loss~Percentage~*~Average~Loss~Per~Losing~Trade} $$

Ideally, a profit factor greater than 1 is desired. Anything below 1 is considered unsatisfactory performance. There is a grading system for the profit factor to help you analyse the performance of your strategy. 

|S.No | Profit Factor | Interpretation    |
|---:|:-------------|:-----------|
| 1 | Below 1  | Strategy is unprofitable |
| 2 | Equal to 1  | Capital at the time of exit is same as capital at time of entry | 
| 3 | Between 1.10 and 1.40 | Strategy provides average returns, but may not withstand high volatility | 
| 4 | Between 1.40 and 2.0 | Strategy is decent | 
| 5 | Equal to or greater than 2  | Strategy is excellent | 

In [9]:
# Calculate profit factor and store in 'analytics'
analytics['Profit Factor'] = (analytics['Win_Percentage']*analytics['per_trade_profit_winners']) / \
    (analytics['Loss_Percentage']*analytics['per_trade_loss_losers'])

# Print the 'analytics' dataframe
round(analytics.T, 2)

,ML_Model
Total_PnL,43223.00
Total_Trades,268.00
Winners,153.00
Losers,115.00
Win_Percentage,57.09
Loss_Percentage,42.91
per_trade_profit_winners,2084.05
per_trade_loss_losers,2143.13
Average holding time,3.00
Profit Factor,1.29


### Save the Files on Your Disk

You can run the following code cell in order to store the trade details into a CSV file. We will be importing the same in the upcoming notebook.

<b>The following cell will not run in the browser. Download this notebook and convert the cell to "Code" type.</b>

## Conclusion
In this notebook, we learned how to use a few trading metrics to analyse the trades generated for the bull call spread strategy based on ML. These analytics help you find how the strategy has performed after the trade has been executed. In the upcoming sections, we will see if we can improve the results of our strategy further by enhancing the ML model.